In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point
import heapq

# Parameters
L = 1.0  # length of the arm
pipe_radius = 0.1
pipe_centers = [(1.0, 1.0), (1.5, 1.0), (2.0, 1.0)]

# Workspace bounds
x_vals = np.linspace(0, 3, 100)
alpha_vals = np.linspace(0, np.pi/2, 100)

# Create configuration space grid
config_space = np.zeros((len(alpha_vals), len(x_vals)), dtype=bool)

# Compute configuration space
for i, alpha in enumerate(alpha_vals):
    for j, x in enumerate(x_vals):
        arm_start = (x, 0)
        arm_end = (x + L * np.cos(alpha), L * np.sin(alpha))
        arm_line = LineString([arm_start, arm_end])
        collision = any(arm_line.distance(Point(px, py)) <= pipe_radius for (px, py) in pipe_centers)
        config_space[i, j] = collision

# Display configuration space
plt.figure(figsize=(10, 6))
plt.imshow(config_space, extent=[x_vals[0], x_vals[-1], np.degrees(alpha_vals[0]), np.degrees(alpha_vals[-1])],
           origin='lower', cmap='Greys', aspect='auto')
plt.xlabel('x (position)')
plt.ylabel('alpha (degrees)')
plt.title('Configuration Space (black = collision)')
plt.grid(True)
plt.show()


In [ ]:
# Utility functions for A* search
def heuristic(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def neighbors(pos, config_space):
    i, j = pos
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    result = []
    for di, dj in moves:
        ni, nj = i + di, j + dj
        if 0 <= ni < config_space.shape[0] and 0 <= nj < config_space.shape[1]:
            if not config_space[ni, nj]:  # Free space only
                result.append((ni, nj))
    return result

def a_star(config_space, start, goal):
    open_set = []
    heapq.heappush(open_set, (0 + heuristic(start, goal), 0, start, [start]))

    visited = set()

    while open_set:
        est_total_cost, cost_so_far, current, path = heapq.heappop(open_set)

        if current in visited:
            continue
        visited.add(current)

        if current == goal:
            return path

        for neighbor in neighbors(current, config_space):
            if neighbor not in visited:
                new_cost = cost_so_far + heuristic(current, neighbor)
                est_total = new_cost + heuristic(neighbor, goal)
                heapq.heappush(open_set, (est_total, new_cost, neighbor, path + [neighbor]))

    return None  # No path found

# Convert physical bin positions to indices in C-space grid
def find_nearest_index(x_target, alpha_target_deg, x_vals, alpha_vals):
    j = (np.abs(x_vals - x_target)).argmin()
    i = (np.abs(np.degrees(alpha_vals) - alpha_target_deg)).argmin()
    return (i, j)

# Red bin -> start config, Blue bin -> goal config
start_pos = find_nearest_index(0.9, 60, x_vals, alpha_vals)
goal_pos = find_nearest_index(2.0, 60, x_vals, alpha_vals)

# Run A* algorithm
path = a_star(config_space, start_pos, goal_pos)

# Plot path on configuration space
plt.figure(figsize=(10, 6))
plt.imshow(config_space, extent=[x_vals[0], x_vals[-1], np.degrees(alpha_vals[0]), np.degrees(alpha_vals[-1])],
           origin='lower', cmap='Greys', aspect='auto')
if path:
    path_x = [x_vals[j] for i, j in path]
    path_alpha = [np.degrees(alpha_vals[i]) for i, j in path]
    plt.plot(path_x, path_alpha, 'r-', linewidth=2, label='Path')
    plt.scatter(x_vals[start_pos[1]], np.degrees(alpha_vals[start_pos[0]]), color='green', label='Start')
    plt.scatter(x_vals[goal_pos[1]], np.degrees(alpha_vals[goal_pos[0]]), color='blue', label='Goal')
    plt.legend()
else:
    plt.title("No path found")
plt.xlabel('x (position)')
plt.ylabel('alpha (degrees)')
plt.title('Configuration Space with Path')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Convert path indices to actual (x, alpha) values
path_values = [(x_vals[j], alpha_vals[i]) for i, j in path]

# Pipe drawing utility
def draw_pipes(ax):
    for (px, py) in pipe_centers:
        pipe = plt.Circle((px, py), pipe_radius, color='gray')
        ax.add_patch(pipe)

# Animation function
def animate_workspace_path(path_values, L):
    fig, ax = plt.subplots(figsize=(8, 4))

    def update(frame):
        ax.clear()
        ax.set_xlim(0, 3.5)
        ax.set_ylim(0, 2)
        ax.set_title('Robot Arm Workspace Animation')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.grid(True)

        # Draw pipes
        draw_pipes(ax)

        # Draw red and blue bins
        ax.add_patch(plt.Rectangle((0.85, 1.2), 0.1, 0.05, color='red'))
        ax.add_patch(plt.Rectangle((1.95, 1.2), 0.1, 0.05, color='blue'))

        # Get robot configuration
        x, alpha = path_values[frame]
        arm_start = (x, 0)
        arm_end = (x + L * np.cos(alpha), L * np.sin(alpha))

        # Draw arm
        ax.plot([arm_start[0], arm_end[0]], [arm_start[1], arm_end[1]], 'b-', linewidth=4)
        ax.plot(arm_start[0], arm_start[1], 'ko')  # base
        ax.plot(arm_end[0], arm_end[1], 'ro')      # end effector

    ani = animation.FuncAnimation(fig, update, frames=len(path_values), interval=200)
    plt.close(fig)
    return ani

# Create the animation
workspace_animation = animate_workspace_path(path_values, L)

# Display in notebook
from IPython.display import HTML
HTML(workspace_animation.to_jshtml())


In [ ]:
# Save the animation as a GIF
gif_path = "/mnt/data/robot_workspace_animation.gif"
workspace_animation.save(gif_path, writer='pillow', fps=5)
gif_path
